In [ ]:
ANTHROPIC_API_KEY = "KEY"
PINECONE_API_KEY = "KEY"
PINECONE_ENVIRONMENT = "us-east-1" # e.g., "us-west1-gcp"
PINECONE_INDEX_NAME = "dense-index"

In [15]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)

# To get the unique host for an index, 
# see https://docs.pinecone.io/guides/data/target-an-index
index = pc.Index(host="https://doc-index-wky5uiz.svc.aped-4627-b74a.pinecone.io")

results = index.search(
    namespace="abudhabi", 
    query={
        "inputs": {"text": "how much percentage  of the water consumed in abu dhabi is ground water"}, 
        "top_k": 5
    },
    fields=[ "page_content"]
)



In [16]:
def get_ctx(query):
    results = index.search(
        namespace="abudhabi", 
        query={
            "inputs": {"text": query}, 
            "top_k": 5
        },
        fields=[ "page_content"]
    )
    
    ctx=[hit['fields']['page_content'] for hit in results['result']['hits']]
    return ctx

In [17]:
get_ctx("how much percentage  of the water consumed in abu dhabi is ground water")

['measurement and limitation of abstracted groundwater quantities. The activities \nare embedded in an overall framework to achieve sustainable groundwater  \nmanagement in Abu Dhabi Emirate.\nStill, 64% of all water used across the Abu Dhabi Emirate is groundwater; that is about  \n2 billion m³/year. Agricultural and forestry sectors account for 95% of these abstractions. \nThe EAD work in the groundwater sector aims to reduce the overall abstraction and to \npreserve the groundwater quality for many years to come. EAD educates groundwater \nusers as well as the general public on groundwater related topics. Hopefully, the \ngroundwater will remain the precious resource for the coming generations that it was \nfor our ancestors.\nDrip Irrigation System\nGROUNDWATER ATLAS OF ABU DHABI EMIRATE\n19',
 '93\nGROUNDWATER ATLAS OF ABU DHABI EMIRATE\n3%\nFresh\nShallow Aquifer\nWest\nShallow Aquifer West \nTOTAL 532 km 3\nShallow Aquifer East \nTOTAL 74 km 3\nData Sources: GWAP 2005, EAD-WSI 2

In [22]:
import anthropic
import os
client = anthropic.Anthropic(
    api_key=ANTHROPIC_API_KEY
)
def call_claude(msg):    
    ctx=get_ctx(msg)
    user_input=f"Query: {msg} \n Context: {ctx}"
    message = client.messages.create(
                model="claude-3-7-sonnet-20250219",
                max_tokens=20000,
                temperature=1,
                system="You are Automated bot to help users get information about abu dhabi , you can make use of the context provided by the user to answer the queries. Any information you provide might or might not be accurate. dont go out of context to answer the query ",
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": user_input
                            }
                        ]
                    }
                ]
            )
    return message

In [25]:
response=call_claude("how much percentage  of the water consumed in abu dhabi is ground water")

In [30]:
if response.stop_reason=="end_turn":
        for item in response.content:
            if item.type == "text":
                  print(item.text)

According to the provided context, 64% of all water used across the Abu Dhabi Emirate is groundwater, which amounts to about 2 billion cubic meters per year. The agricultural and forestry sectors account for 95% of these groundwater abstractions.
